In [8]:
import re
from bs4 import BeautifulSoup
from mechanize import Browser
import pandas as pd
import requests
from IPython.display import display_html
from tqdm import tqdm

br = Browser()

br.set_handle_robots( False )
br.addheaders = [("User-agent", "Firefox")]
br.open("http://sistemas.opds.gba.gov.ar/intra/Profesionales/ConsultaWeb.php")

br.select_form("frmBuscaProf")
controls = br.form.controls
print([control.name for control in controls])

['sltRegistro', 'txtNroProf', 'txtDNIBusca', 'txtApellidoBusca', 'txtNombre', 'sltTitulo', 'sltProvincia', 'sltPartidoR', 'sltLocalidadR', 'btnBuscaDNI']


In [3]:

url="http://sistemas.opds.gba.gov.ar/intra/Profesionales/ConsultaWeb.php"
html_content = requests.get(url).text
soup = BeautifulSoup(html_content, "lxml")
options = soup.find_all("option")
for k, i in enumerate(options):
    print(k,"   ", i,"   value: ", i.get("value"))

0     <option value="0">-- Seleccione Registro --</option>    value:  0
1     <option value="3">Consultoras de Evaluación</option>    value:  3
2     <option value="1">Profesionales de Aparatos Sometidos A Presión</option>    value:  1
3     <option value="2">Profesionales de Evaluación</option>    value:  2
4     <option value="13">Técnicos de Aparatos Sometidos a Presión</option>    value:  13
5     <option value="nada">-- Seleccione Título --</option>    value:  nada
6     <option value="	LIC. SEGURIDAD E HIGIENE Y CONTROL AMBIENTAL ">	LIC. SEGURIDAD E HIGIENE Y CONTROL AMBIENTAL </option>    value:  	LIC. SEGURIDAD E HIGIENE Y CONTROL AMBIENTAL 
7     <option value="ABOGADA">ABOGADA</option>    value:  ABOGADA
8     <option value="Abogado">Abogado</option>    value:  Abogado
9     <option value="Agrimensor">Agrimensor</option>    value:  Agrimensor
10     <option value="Agrónomo">Agrónomo</option>    value:  Agrónomo
11     <option value="Analista de Sistemas">Analista de Sistemas<

In [7]:
print(html_content)

  <html>
<head>
<title>Registro de Profesionales</title>
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1">
<link rel="stylesheet" type="text/css" href="../EstiloNuevo/general.css" />



</head>

	<script>
	function solonumeros() {
		if ((event.keyCode < 48) || (event.keyCode > 57)) 
		{	
			event.returnValue = false;
		}
	} // end function 


function validar(){
	   mensaje='';

    var indice = document.formMovim.sltAreas.selectedIndex;  
    var tipoarea = document.formMovim.sltAreas.options[indice].value;
    if (tipoarea == ''){ mensaje='Debe seleccionar el Area del cual decea Recibir los Expedientes'; } 
	if (formMovim.numRemito.value == ''){ mensaje='Debe seleccionar un Remito'; }   
    

		  if (mensaje!=''){
  			  alert(mensaje);
		     return false;
		  }else{ return true;}
	}


</script>
<body>
<p>
    <br>
</p>
<table width="1100" border="0" align="center" class="tabla">
  <tr>
    <td><img src="../../Imagenes/header_opds.jpg" width="1100" height="13

In [ ]:
registro = [0,1,2,3]
provincias=[]
partidos=[]

In [4]:
tits = []
for k, i in enumerate(options):
    if (k>5)&(k<390):
        tits.append(i.get("value"))
    if (k>390):
        provincias.append(i.get("value"))

In [14]:
finals = pd.Series()
falied=0
for i in tqdm(tits):
    br = Browser()

    br.set_handle_robots( False )
    br.addheaders = [("User-agent", "Firefox")]
    br.open("http://sistemas.opds.gba.gov.ar/intra/Profesionales/ConsultaWeb.php")

    br.select_form("frmBuscaProf")
    br.form["sltTitulo"] = [str(i),]
    br.submit()
    resp = br.response()
    soup = BeautifulSoup(resp.read(), "html.parser")
    data = soup.find("table", border=1)
    html_string = str(data)
    try:

        dat = pd.read_html(html_string)
        mails = dat[0][6][1:]
        mails = mails.dropna()
        finals = finals.append(mails)
    except Exception:
        failed+=1
        pass
    print(i, "Done", finals)
    finals.to_csv("mails_pais.csv",index=False)


  0%|          | 0/384 [00:00<?, ?it/s]/usr/lib/python3/dist-packages/ipykernel_launcher.py:26: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.

  0%|          | 1/384 [00:06<43:20,  6.79s/it]

	LIC. SEGURIDAD E HIGIENE Y CONTROL AMBIENTAL  Done 1    ggcisterna@gmail.com
dtype: object



  1%|          | 2/384 [00:15<46:15,  7.27s/it]

ABOGADA Done 1          ggcisterna@gmail.com
1    sanchogabriela@hotmail.com
dtype: object



  1%|          | 3/384 [00:39<1:17:53, 12.27s/it]

...

TECNICO UNIVERSITARIO EN MEDIO AMBIENTE Done 1           ggcisterna@gmail.com
1     sanchogabriela@hotmail.com
1         ecalamos@argentina.com
4         conghos@hotmail.com.ar
5    marianonicola1975@gmail.com
                ...             
1      mauromerlo82@yahoo.com.ar
1          sebavines@hotmail.com
1      sergiociglesias@gmail.com
1         diegofcuesta.gmail.com
1     MOLINOS@MOLINOSTSAS.COM.AR
Length: 931, dtype: object



100%|█████████▉| 383/384 [1:13:23<00:08,  8.46s/it]

Veterinario Done 1                ggcisterna@gmail.com
1          sanchogabriela@hotmail.com
1              ecalamos@argentina.com
4              conghos@hotmail.com.ar
5         marianonicola1975@gmail.com
                   ...               
1               sebavines@hotmail.com
1           sergiociglesias@gmail.com
1              diegofcuesta.gmail.com
1          MOLINOS@MOLINOSTSAS.COM.AR
1    jrlamas@laboratoriollamas.com.ar
Length: 932, dtype: object



100%|██████████| 384/384 [1:13:32<00:00,  8.66s/it]

Zoólogo Done 1                ggcisterna@gmail.com
1          sanchogabriela@hotmail.com
1              ecalamos@argentina.com
4              conghos@hotmail.com.ar
5         marianonicola1975@gmail.com
                   ...               
1               sebavines@hotmail.com
1           sergiociglesias@gmail.com
1              diegofcuesta.gmail.com
1          MOLINOS@MOLINOSTSAS.COM.AR
1    jrlamas@laboratoriollamas.com.ar
Length: 932, dtype: object


In [18]:
finals.head(100)

1            ggcisterna@gmail.com
1      sanchogabriela@hotmail.com
1          ecalamos@argentina.com
4          conghos@hotmail.com.ar
5     marianonicola1975@gmail.com
                 ...             
10      ingenierosendin@gmail.com
11      adrianaspagnolo@gmail.com
1           lauccordoba@gmail.com
2          alejandro@falco.net.ar
1       jmgconstrucciones.2@gmail
Length: 100, dtype: object

In [16]:
finals.shape

(932,)